## Для примера импортируем превый датасет с вопросами

In [1]:
import pandas as pd


test_df = pd.read_csv('./quest_data/quest_answer_context_0.csv')

In [72]:
question_test = test_df.iloc[89].question
answer_test = test_df.iloc[89].answer

In [73]:
test_df.iloc[89].context

'Это соответствует 8% от общего объема рынка генеративного искусственного интеллекта с \nдоходом уже в 3,7 миллиарда долларов США. Ожидается, что к 2028 году рынок \nмузыкальных р ешений с ИИ увеличится более чем в десять раз, а среднегодовые темпы роста \nсоставят около 60% . \n• 71% опрошенных авторов музыки во Франции и Германии опасаются, что использование \nискусственного интеллекта в их индустрии может привести к резкому понижению доходов'

In [74]:
question_test

'Какой объем рынка генеративного искусственного интеллекта составляет музыкальные решения с ИИ?'

In [75]:
answer_test

'8% от общего объема рынка.'

## Возьмем созданную базу данных с эмбедингом

In [2]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_chroma import Chroma

model_name = "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
embedding = HuggingFaceEmbeddings(model_name=model_name,
                                  model_kwargs=model_kwargs,
                                  encode_kwargs=encode_kwargs)

/usr/lib/python3.10/importlib/__init__.py:126: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  return _bootstrap._gcd_import(name[level:], package, level)
/tmp/ipykernel_8190/2540701653.py:7: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langcha

In [6]:
vector_store = Chroma(collection_name = '500100', embedding_function = embedding, persist_directory = './DataBase/db_500_100')

In [82]:
result_question = [i.page_content for i in vector_store.similarity_search(question_test, k = 100)]

## Функция для выделения релевантности эмбеддинга
строим новый вектор стор для ответа и эмбеддингов и выбираем k ближайших к нему эмбеддингов, кто из results попал в данный список , считаются релевантными остальные нет.

In [83]:
import string
from langchain.retrievers import BM25Retriever 


def tokenize(s):
    return s.lower().translate(str.maketrans("", "", string.punctuation)).split(" ")
     

bm25_retriever = BM25Retriever.from_texts(
      texts=vector_store.get()['documents'],
      preprocess_func=tokenize,
      k=100,
  )
result_answer = [i.page_content for i in bm25_retriever.get_relevant_documents(answer_test)]

In [84]:
result_question_set = set(result_question)
result_answer_set = set(result_answer)
result_cross_set = result_question_set & result_answer_set
result_question_diff_set = result_question_set - result_answer_set

In [85]:
len(result_cross_set)

2

In [117]:
def Rank_precision(table, rank):
    table_new = table.loc[table.Rank <= rank]
    return len(table_new.loc[table_new.Judgment == True]) / len(table_new)


def Rank_recall(table, rank):
    all_R = table.loc[table.Judgment == True]
    if len(all_R) == 0:
        return 0
    table_new = table.loc[table.Rank <= rank]
    return len(table_new.loc[table_new.Judgment == True]) / len(all_R)

In [91]:
rank_table = pd.DataFrame(columns = ['chunk', 'Rank', 'Judgment', 'Precision', 'Recall'])
rank_table['chunk'] = result_question
rank_table['Rank'] = list(range(len(result_question)))
rank_table['Judgment'] = rank_table.chunk.apply(lambda x: x in result_cross_set)

In [107]:
Rank_precision_list = []
Rank_recall_list = []

for r in rank_table.Rank:
    Rank_precision_list.append(Rank_precision(rank_table, r))
    Rank_recall_list.append(Rank_recall(rank_table, r))

rank_table['Precision'] = Rank_precision_list
rank_table['Recall'] = Rank_recall_list


In [108]:
rank_table

,chunk,Rank,Judgment,Precision,Recall
0,11 \n \n Национальный центр развития искусстве...,0,False,0.000000,0.0
1,портала в сфере ИИ \n \n \nЭтот отчет являет...,1,True,0.500000,0.5
2,Это соответствует 8% от общего объема рынка ге...,2,True,0.666667,1.0
3,интеллекта в музыкальном секторе/AI and Music....,3,False,0.500000,1.0
4,G O L D M E D I A\n”Artificial intelligence wi...,4,False,0.400000,1.0
...,...,...,...,...,...
95,Влияние искусственного интеллекта на образован...,95,False,0.020833,1.0
96,Влияние искусственного интеллекта на образован...,96,False,0.020619,1.0
97,G O L D M E D I AEven though only 15% of the s...,97,False,0.020408,1.0
98,"vulnerable than others. ”Generally speaking, t...",98,False,0.020202,1.0


In [120]:
# Average precision
def AP(table):
    choose_relevant = table.loc[table.Judgment == True].Precision
    if choose_relevant.count() == 0:
        return 0
    return choose_relevant.sum() / choose_relevant.count()

In [110]:
AP(rank_table)

0.5833333333333333

## Сделаем метрику на несколько вопросов

In [122]:
import numpy as np
AP_list = []
k = 10
bm25_retriever = BM25Retriever.from_texts(
      texts=vector_store.get()['documents'],
      preprocess_func=tokenize,
      k=k,
  )
for i in range(len(test_df)):
    question_test = test_df.iloc[i].question
    answer_test = test_df.iloc[i].answer
    result_question = [i.page_content for i in vector_store.similarity_search(question_test, k = k)]
    result_answer = [i.page_content for i in bm25_retriever.get_relevant_documents(answer_test)]
    result_question_set = set(result_question)
    result_answer_set = set(result_answer)
    result_cross_set = result_question_set & result_answer_set
    rank_table = pd.DataFrame(columns = ['chunk', 'Rank', 'Judgment', 'Precision', 'Recall'])
    rank_table['chunk'] = result_question
    rank_table['Rank'] = list(range(len(result_question)))
    rank_table['Judgment'] = rank_table.chunk.apply(lambda x: x in result_cross_set)

    Rank_precision_list = []
    Rank_recall_list = []
    
    for r in rank_table.Rank:
        Rank_precision_list.append(Rank_precision(rank_table, r))
        Rank_recall_list.append(Rank_recall(rank_table, r))
    
    rank_table['Precision'] = Rank_precision_list
    rank_table['Recall'] = Rank_recall_list

    AP_val = AP(rank_table)
    
    AP_list.append(AP_val)
    print('Question: ', question_test, 'with AP = ', AP_val)


Question:  Какие четыре механизма политики используются для увеличения регуляторной видимости? with AP =  0
Question:  Какие проблемы возникают при работе с большим объемом данных? with AP =  0
Question:  Какие страны будут вести экономическую разведку? with AP =  0
Question:  Кто является редактором Munich Security Report 2024? with AP =  0
Question:  Для чего используется синтетическая данные? with AP =  0
Question:  Какие меры предпринимаются для обеспечения соблюдения прав человека в отношении систем искусственного интеллекта? with AP =  0.19642857142857142
Question:  Когда был опубликован выпуск 1, номер 1 журнала? with AP =  0
Question:  Кто стала основоположником первой советской школы теоретического программирования? with AP =  0
Question:  Какие формы кода и артефакты необходимо хранить в системе контроля версий? with AP =  1.0
Question:  Что влияет на количество необходимых компьютерных ресурсов для обучения и работы модели машинного обучения? with AP =  0
Question:  Как созд

In [123]:
print('MAP = ', np.mean(AP_list))

MAP =  0.06850396825396825
